# Creating Database for Analysis

In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [3]:
#csv files to read into dataframe
mcsvfile = "clean_hawaii_measurements.csv"
mdf = pd.read_csv(mcsvfile, dtype=object)
mdf.head()

,id,station,date,prcp,tobs
0,1,USC00519397,1/1/2010,0.08,65
1,2,USC00519397,1/2/2010,0.0,63
2,3,USC00519397,1/3/2010,0.0,74
3,4,USC00519397,1/4/2010,0.0,76
4,6,USC00519397,1/7/2010,0.06,70


In [4]:
scsvfile = "clean_hawaii_stations.csv"
sdf = pd.read_csv(scsvfile, dtype=object)
sdf.head()

,station_id,name,latitude,longitude,elevation
0,USC00519397,WAIKIKI 717.2,21.2716,-157.8168,3
1,USC00513117,KANEOHE 838.1,21.4234,-157.8015,14.6
2,USC00514830,KUALOA RANCH HEADQUARTERS 886.9,21.5213,-157.8374,7
3,USC00517948,PEARL CITY,21.3934,-157.9751,11.9
4,USC00518838,UPPER WAHIAWA 874.3,21.4992,-158.0111,306.6


In [17]:
# Create an engine to a SQLite database
engine = create_engine("sqlite:///hawaii.sqlite")

In [18]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [19]:
# Use `declarative_base` from SQLAlchemy to model the measurement and station tables as an ORM class
# specify types for each column, e.g. Integer, Text, etc.

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Station(Base):
    __tablename__ = 'stations'

    station_id = Column(Text, primary_key=True)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [20]:
# Use `create_all` to create tables in the database
Base.metadata.create_all(engine)

In [21]:
# Use Orient='records' to create a list of data to write
mdata = mdf.to_dict(orient='records')
sdata = sdf.to_dict(orient='records')

In [10]:
print(mdata[:3])

[{'id': '1', 'station': 'USC00519397', 'date': '1/1/2010', 'prcp': '0.08', 'tobs': '65'}, {'id': '2', 'station': 'USC00519397', 'date': '1/2/2010', 'prcp': '0.0', 'tobs': '63'}, {'id': '3', 'station': 'USC00519397', 'date': '1/3/2010', 'prcp': '0.0', 'tobs': '74'}]


In [11]:
print(sdata[:3])

[{'station_id': 'USC00519397', 'name': 'WAIKIKI 717.2', 'latitude': '21.2716', 'longitude': '-157.8168', 'elevation': '3'}, {'station_id': 'USC00513117', 'name': 'KANEOHE 838.1', 'latitude': '21.4234', 'longitude': '-157.8015', 'elevation': '14.6'}, {'station_id': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9', 'latitude': '21.5213', 'longitude': '-157.8374', 'elevation': '7'}]


In [22]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [28]:
# Save the reference to tables as a variables
stable = sqlalchemy.Table('stations', metadata, autoload=True)
mtable = sqlalchemy.Table('measurements', metadata, autoload=True)

In [25]:
# Insert the data into each of the tables
# The SQL table is populated during this step
conn.execute(mtable.insert(), mdata)

In [26]:
conn.execute(stable.insert(), sdata)